In [1]:
import os
import import_ipynb
import pandas as pd
import itertools
import talib
import math
import numpy as np

'''to be modified and shift into property files'''
currentpath=os.getcwd()
dbpath=os.path.join(currentpath,"..",'database')
stockcsv=os.path.join(dbpath,'stockpath.csv')
featurescsv=os.path.join(dbpath,'features.csv')
featuresdata=os.path.join(dbpath,'featuresdata.csv')



#to import Load_Csv file from another directory. this style is for jupyter notebook
                
%cd ..
from utility import Load_Csv as lcsv
%cd stockprediction



C:\Users\shaskhan\AnacondaProjects\am\finProduct
importing Jupyter notebook from C:\Users\shaskhan\AnacondaProjects\am\finProduct\utility\Load_Csv.ipynb
C:\Users\shaskhan\AnacondaProjects\am\finProduct\stockprediction


In [15]:
class ta(lcsv.Load_csv):
    '''this class contains functions fto predict label with the help of technical indicators'''

    def loadcsv(self):
        '''load 'Date','Close', 'Volume' data from databse and return dataframe
        '''
        
        self.dataset=self.LoadData(stockcsv).loc[:, ['Date','Close', 'Volume']]
        self.dataset['Date'] = pd.to_datetime(self.dataset['Date'])
        return self.dataset
    

    def loadfeaturesdata(self,x):
        '''load features detailed data from databse'''
        self.featuresdata=self.LoadfeaData(featuresdata)
        self.featuresdata.index=self.featuresdata[0]
        #self.featuresdata=self.featuresdata.loc[x]
        return list(self.featuresdata.loc[x,1:])
    
    
    def ti_Combinations(self,a):
        '''takes input as list of list and  gives output as comninations '''

        df=pd.Series(list(itertools.product(*a)))
        return df
    
        
        
    def loadfeatures(self):
        '''load feature label data from databse
        '''
        self.featurestilist=[]
        self.misc=[]
        self.label=[]
        self.tilist=dir(talib)
        self.featuredict={}
        self.features=self.LoadfeaData(featurescsv)

        def fun(row):
            r_len=len(row)
            
                     
            if row[0] in self.tilist:
                self.featuredict[row[0]]=[]
                self.featurestilist.append(row[0])
                
                for i in range(r_len-1):                       
                        
                    if str(row[i+1]).find("-")>-1:   #tocheck if range is given 
                        a,b=row[i+1].split("-")
                        a=int(a)
                        b=int(b)
                        
                        self.featuredict[row[0]].append(list(range(a,b)))   #if range is given then replace it by list
                    #elif math.isnan(row[i+1]):                              # ignore if blank found
                        #continue
                    else:
                        try:
                            var=str(row[0])+'-'+str(int(row[i+1]))              #togenerate name like MA-1,MA-2  

                            self.featuredict[row[0]].extend(self.loadfeaturesdata(var))
                        except Exception as e:
                            print('e1',e)
                        
                print(self.featuredict) #To be removed after development
                
            elif row[0]=='label':
                self.label.append(row[1])
            else:
                self.misc.append(row[0])
                
            try:
                a=self.featuredict[row[0]]
                #print(a)
                #self.MA_comb=(self.ti_Combinations(a))    ##get combinatons 
            except Exception as e:
                print('e2',e)
            
            
                
        #vectorize approach to speed up process        
        self.features.apply(fun, axis=1)
        #print("end")
        
        
        
        #return(self.featurestilist,self.label,self.misc,self.MA_comb)
        
        
        
    def get_MA(self):
        malist=self.featuredict['MA']
        for i in malist:
            
            self.dataset['MA'+str(i)]=talib.SMA(self.dataset['Close'],i)
        print(self.dataset.tail)
        
    def get_RSI(self):
        rsilist=self.featuredict['RSI']
        for i in rsilist:
            
            self.dataset['RSI'+str(i)]=talib.SMA(self.dataset['Close'],30)
        print(self.dataset.tail)
        
        
        
    def get_technical_indi(self):
        
        for i in self.featurestilist:
            if i=='MA':
                self.get_MA()
            if i=='RSI':
                self.get_RSI()
        
        
            
            
            #df['SMA']= df['close'].rolling(window=n).mean()
            
            
            #df['RSI']=ta.RSI(np.array(df['close']), timeperiod=n)

            

            
    
    
    

    
  
        
        
        

In [16]:

b=ta()
b.loadcsv()
b.loadfeatures()
b.get_technical_indi()



{'MA': [10, 50, 60]}
e1 cannot convert float NaN to integer
{'MA': [10, 50, 60], 'RSI': [1, 10]}
e2 'label'
<bound method NDFrame.tail of          Date     Close       Volume       MA10       MA50          MA60
0  2018-01-01  10435.55  134532090.0        NaN        NaN           NaN
1  2018-01-02  10442.20  158092430.0        NaN        NaN           NaN
2  2018-01-03  10443.20  172516859.0        NaN        NaN           NaN
3  2018-01-04  10504.80  180257392.0        NaN        NaN           NaN
4  2018-01-05  10558.85  186469717.0        NaN        NaN           NaN
5  2018-01-08  10623.60  174181231.0        NaN        NaN           NaN
6  2018-01-09  10637.00  211291563.0        NaN        NaN           NaN
7  2018-01-10  10632.20  181900014.0        NaN        NaN           NaN
8  2018-01-11  10651.20  158630055.0        NaN        NaN           NaN
9  2018-01-12  10681.25  180592153.0  10560.985        NaN           NaN
10 2018-01-15  10741.55  181262074.0  10591.585        NaN 